Scanning the area, you discover that the entire field you're standing on is densely packed with pipes; it was hard to tell at first because they're the same metallic silver color as the "ground". You make a quick sketch of all of the surface pipes you can see (your puzzle input).

The pipes are arranged in a two-dimensional grid of tiles:

    | is a vertical pipe connecting north and south.  
    - is a horizontal pipe connecting east and west.  
    L is a 90-degree bend connecting north and east.  
    J is a 90-degree bend connecting north and west.  
    7 is a 90-degree bend connecting south and west.  
    F is a 90-degree bend connecting south and east.  
    . is ground; there is no pipe in this tile.  
    S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.  

Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is one large, continuous loop.
Find the single giant loop starting at S. How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?

In [25]:
pipe_table = {'north': {'|': (1, 0, 'north'),
                        'J': (0, -1, 'east'),
                        'L': (0, 1, 'west')},
              'south': {'|': (-1, 0, 'south'),
                        '7': (0, -1, 'east'),
                        'F': (0 , 1, 'west')},
              'east': {'-': (0, -1, 'east'),
                       'F': (1, 0, 'north'),
                       'L': (-1, 0, 'south')},
              'west': {'-': (0, 1, 'west'),
                       'J': (-1, 0 , 'south'),
                       '7': (1, 0, 'north')}}

grid = []
with open('input.txt') as file:
    for line in file:
        grid.append(line.strip())

num_rows = len(grid)
num_cols = len(grid[0])

found = False
for r in range(num_rows):
    for c in range(num_cols):
        if grid[r][c] == 'S':
            row = r
            col = c
            found = True
            break
    if found:
        break     

if (row-1 >= 0) and (grid[row-1][col] in {'|', '7', 'F'}):
    pos = grid[row-1][col]
    prev = 'south'
    row -= 1
elif (row+1 < num_rows) and (grid[row+1][col] in {'|', 'J', 'L'}):
    pos = grid[row+1][col]
    prev = 'north'
    row += 1
elif (col-1 >= 0) and (grid[row][col-1] in {'-', 'F', 'L'}):
    pos = grid[row][col-1]
    prev = 'east'
    col -= 1
elif (col+1 < num_cols) and (grid[row][col+1] in {'-', 'J', '7'}):
    pos = grid[row][col+1]
    prev = 'west'
    col += 1

steps = 0
while True:
    if pos == 'S':
        break
    else:
        r, c, fm = pipe_table[prev][pos]
        row = row + r
        col = col + c
        prev = fm
        pos = grid[row][col]
        steps += 1

print(steps//2 + 1)

6897


Figure out whether you have time to search for the nest by calculating the area within the loop. How many tiles are enclosed by the loop?

In [14]:
pipe_table = {'north': {'|': (1, 0, 'north'),
                        'J': (0, -1, 'east'),
                        'L': (0, 1, 'west')},
              'south': {'|': (-1, 0, 'south'),
                        '7': (0, -1, 'east'),
                        'F': (0 , 1, 'west')},
              'east': {'-': (0, -1, 'east'),
                       'F': (1, 0, 'north'),
                       'L': (-1, 0, 'south')},
              'west': {'-': (0, 1, 'west'),
                       'J': (-1, 0 , 'south'),
                       '7': (1, 0, 'north')}}

joint_pairs = {('F', '7'): 0,
               ('F', 'J'): 1,
               ('L', 'J'): 0,
               ('L', '7'): 1}

s_replace = {('south', 'south'): '|',
             ('south', 'east'): 'L',
             ('south', 'west'): 'J',
             ('north', 'north'): '|',
             ('north', 'east'): 'F',
             ('north', 'west'): '7',
             ('east','east'): '-',
             ('east', 'north'): 'J',
             ('east','south'): '7',
             ('west', 'west'): '-',
             ('west', 'north'): 'L',
             ('west', 'south'): 'F'}

grid = []
with open('input.txt') as file:
    for line in file:
        grid.append(line.strip())

num_rows = len(grid)
num_cols = len(grid[0])

found = False
for r in range(num_rows):
    for c in range(num_cols):
        if grid[r][c] == 'S':
            row = r
            col = c
            found = True
            break
    if found:
        break     

pipe_pos = {(row, col)}

if (row-1 >= 0) and (grid[row-1][col] in {'|', '7', 'F'}):
    pos = grid[row-1][col]
    prev = 'south'
    row -= 1
elif (row+1 < num_rows) and (grid[row+1][col] in {'|', 'J', 'L'}):
    pos = grid[row+1][col]
    prev = 'north'
    row += 1
elif (col-1 >= 0) and (grid[row][col-1] in {'-', 'F', 'L'}):
    pos = grid[row][col-1]
    prev = 'east'
    col -= 1
elif (col+1 < num_cols) and (grid[row][col+1] in {'-', 'J', '7'}):
    pos = grid[row][col+1]
    prev = 'west'
    col += 1

start_dir = prev

while True:
    if pos == 'S':
        s = list(grid[row])
        s[col] = s_replace[(start_dir, prev)]
        grid[row] = ''.join(s)
        break
    else:
        pipe_pos.add((row, col))
        r, c, fm = pipe_table[prev][pos]
        row = row + r
        col = col + c
        prev = fm
        pos = grid[row][col]

def check_pip(row, col):
    count = 0
    entry_joint = ''
    for i in range(col, num_cols):
        if (row, i) in pipe_pos:
            symbol = grid[row][i]
            if symbol == '|':
                count += 1
            elif symbol == 'F' or symbol == 'L':
                entry_joint = symbol
            elif symbol == 'J' or symbol == '7':
                count += joint_pairs[(entry_joint, symbol)]
    if count%2:
        return 1
    else:
        return 0

area = 0
for i in range(num_rows):
    for j in range(num_cols-1):
        if (i, j) not in pipe_pos:
            area += check_pip(i, j+1)
print(area)

367
